# Import Needed Package

In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Once your drive is mounted, you can load your data. For example, if you have a CSV file named `my_data.csv` in the root of your Google Drive, you can load it using pandas:

In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/pertemuan 10 /IMDB Dataset.csv")

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [ ]:
data.shape

(50000, 2)

In [ ]:
data.tail()

,review,sentiment
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative
49999,No one expects the Star Trek movies to be high...,negative


In [ ]:
data["sentiment"].value_counts()

,count
sentiment,
positive,25000
negative,25000


In [ ]:
data.replace({"sentiment": {"positive":1, "negative":0}}, inplace=True)

In [ ]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [ ]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

In [ ]:
train_data.shape

(40000, 2)

In [ ]:
test_data.shape

(10000, 2)

In [ ]:
tokenizer = Tokenizer(num_words = 5000)
tokenizer.fit_on_texts(train_data["review"])

In [ ]:
x_train = pad_sequences(tokenizer.texts_to_sequences(train_data["review"]), maxlen=200)
x_test = pad_sequences(tokenizer.texts_to_sequences(test_data["review"]), maxlen=200)

In [ ]:
x_train

array([[1935,    1, 1200, ...,  205,  351, 3856],
       [   3, 1651,  595, ...,   89,  103,    9],
       [   0,    0,    0, ...,    2,  710,   62],
       ...,
       [   0,    0,    0, ..., 1641,    2,  603],
       [   0,    0,    0, ...,  245,  103,  125],
       [   0,    0,    0, ...,   70,   73, 2062]], dtype=int32)

In [ ]:
x_test

array([[   0,    0,    0, ...,  995,  719,  155],
       [  12,  162,   59, ...,  380,    7,    7],
       [   0,    0,    0, ...,   50, 1088,   96],
       ...,
       [   0,    0,    0, ...,  125,  200, 3241],
       [   0,    0,    0, ..., 1066,    1, 2305],
       [   0,    0,    0, ...,    1,  332,   27]], dtype=int32)

In [ ]:
Y_train = train_data["sentiment"]
Y_test = test_data["sentiment"]

In [ ]:
Y_train

,sentiment
39087,0
30893,0
45278,1
16398,0
13653,0
...,...
11284,1
44732,1
38158,0
860,1


# Model Building

In [ ]:
model = Sequential()
model.add(Embedding(input_dim = 5000, output_dim = 128, input_length=200))
model.add(LSTM(units=128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation="sigmoid"))

In [ ]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [ ]:
model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

In [ ]:
model.fit(x_train, Y_train, batch_size=64, epochs=5, validation_split = 0.2)

Epoch 1/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 208s 399ms/step - accuracy: 0.7228 - loss: 0.5290 - val_accuracy: 0.7937 - val_loss: 0.4632
Epoch 2/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 400ms/step - accuracy: 0.8290 - loss: 0.3934 - val_accuracy: 0.8471 - val_loss: 0.3966
Epoch 3/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 200s 401ms/step - accuracy: 0.8603 - loss: 0.3340 - val_accuracy: 0.8310 - val_loss: 0.3906
Epoch 4/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 201s 401ms/step - accuracy: 0.8934 - loss: 0.2693 - val_accuracy: 0.8737 - val_loss: 0.3266
Epoch 5/5
500/500 ━━━━━━━━━━━━━━━━━━━━ 215s 430ms/step - accuracy: 0.9140 - loss: 0.2222 - val_accuracy: 0.8710 - val_loss: 0.3255


In [ ]:
loss, accuracy  = model.evaluate(x_test, Y_test)
print(f"Test Loss: {loss:.4f}")
print(f"Test Accuracy: {accuracy:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 38s 120ms/step - accuracy: 0.8711 - loss: 0.3168
Test Loss: 0.3128
Test Accuracy: 0.8737


predictive system


In [ ]:
def predictive_system(review0):
  sequences = tokenizer.texts_to_sequences([review0])
  padded_sequence = pad_sequences(sequences, maxlen=200)
  prediction = model.predict(padded_sequence)
  sentiment = "positive" if prediction > 0.5 else "negative"
  return sentiment


In [ ]:
predictive_system("This movie was ass")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 136ms/step


'negative'

In [ ]:
predictive_system("This movie was visually award worthy")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step


'positive'

In [ ]:
predictive_system("This movie is negative")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'positive'

In [ ]:
predictive_system("This movie is positive")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 129ms/step


'positive'

save model


In [ ]:
model.save("model.h5")


In [ ]:
import joblib
joblib.dump(tokenizer, "tokenizer.pkl")

['tokenizer.pkl']